In [ ]:
# Focal Loss with class weights
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.ce = nn.CrossEntropyLoss(reduction='none')

    def forward(self, inputs, targets):
        ce_loss = self.ce(inputs, targets)
        pt = torch.exp(-ce_loss)
        if self.alpha is not None:
            at = self.alpha[targets]
            loss = at * (1 - pt) ** self.gamma * ce_loss
        else:
            loss = (1 - pt) ** self.gamma * ce_loss
        return loss.mean()
# Device & training
best_f1 = 0.0
pathbestmodel = ''
from sklearn.metrics import f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DenseNet201_CBAM(num_classes=len(classes)).to(device)
alpha_weights = torch.tensor([1.2, 1.0, 2.5, 1.2, 3.0], dtype=torch.float32).to(device)
criterion = FocalLoss(alpha=alpha_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

train_losses, valid_losses, accuracies = [], [], []
num_epochs = 10
count=0
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_dataset_clear_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_dataset_clear_loader.dataset)
    train_losses.append(epoch_loss)

    model.eval()
    valid_loss, correct = 0.0, 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    valid_epoch_loss = valid_loss / len(valid_loader.dataset)
    accuracy = correct.double() / len(valid_loader.dataset)
    f1 = f1_score(all_labels, all_preds, average='macro')
    valid_losses.append(valid_epoch_loss)
    accuracies.append(accuracy.item())

    print(f"Phase 1: Epoch {epoch+1}: Train Loss = {epoch_loss:.4f}, Val Loss = {valid_epoch_loss:.4f}, Acc = {accuracy:.4f}, F1 Score(macro) = {f1:.4f} ")
    if f1 > best_f1:
        best_f1 = f1
        pathbestmodel = f'/content/drive/MyDrive/Colab Notebooks/model/model_test/Densenet_CBAM_with_Xsign_clear{count}.pt'
        torch.save(model.state_dict(), pathbestmodel)
        count += 1
        print("✅ Saved best model!")